## Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import gensim
from gensim import corpora
from gensim.models import LsiModel

## Read cleaned data files

In [2]:
cleaned_articles_df = pd.read_csv("cleaned_articles.csv")

In [3]:
cleaned_articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42322 entries, 0 to 42321
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   publish_date     42322 non-null  object
 1   title            42322 non-null  object
 2   author           42322 non-null  object
 3   url              42322 non-null  object
 4   claps            42322 non-null  int64 
 5   responses        42322 non-null  int64 
 6   reading_time     42322 non-null  int64 
 7   paid             42322 non-null  int64 
 8   content          42322 non-null  object
 9   cleaned_content  42232 non-null  object
 10  cleaned_author   42322 non-null  object
dtypes: int64(4), object(7)
memory usage: 3.6+ MB


In [4]:
cleaned_jobs_listings_df = pd.read_csv("cleaned_jobs_listings.csv")

In [5]:
cleaned_jobs_listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3909 entries, 0 to 3908
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   job_title                3909 non-null   object 
 1   salary_estimate          3909 non-null   object 
 2   job_description          3909 non-null   object 
 3   rating                   3909 non-null   float64
 4   company_name             3909 non-null   object 
 5   location                 3909 non-null   object 
 6   headquarters             3909 non-null   object 
 7   size                     3909 non-null   object 
 8   founded                  3909 non-null   int64  
 9   type_of_ownership        3909 non-null   object 
 10  industry                 3909 non-null   object 
 11  sector                   3909 non-null   object 
 12  revenue                  3909 non-null   object 
 13  competitors              3909 non-null   object 
 14  easy_apply              

## Data Preprocessing

In [6]:
new_stopwords = ['job', 'skill', 'experience', 'team', 'data', 'use', 'like', 'business', 'work', 'ability', 'let', 'example'
                'need', 'new', 'user', 'opportunity', 'candidate', 'provide', 'company']

In [7]:
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='n')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and token not in new_stopwords:
            result.append(lemmatize(token))
    return result

In [8]:
preprocessed_articles = cleaned_articles_df['cleaned_content'].fillna('').astype(str).map(preprocess)

In [9]:
articles_dictionary = corpora.Dictionary(preprocessed_articles)

In [10]:
articles_corpus = [articles_dictionary.doc2bow(text) for text in preprocessed_articles]

In [11]:
preprocessed_jobs_listings = cleaned_jobs_listings_df['cleaned_job_description'].fillna('').astype(str).map(preprocess)

In [12]:
jobs_listings_dictionary = corpora.Dictionary(preprocessed_jobs_listings)

In [13]:
jobs_listings_corpus = [jobs_listings_dictionary.doc2bow(text) for text in preprocessed_jobs_listings]

## LSA Model

In [14]:
articles_lsamodel = LsiModel(articles_corpus, num_topics=10, id2word = articles_dictionary)

In [15]:
articles_lsamodel.print_topics(num_words=5)

[(0,
  '0.456*"model" + 0.195*"learning" + 0.178*"value" + 0.159*"function" + 0.155*"time"'),
 (1,
  '-0.723*"model" + 0.143*"function" + 0.131*"time" + 0.124*"need" + -0.121*"training"'),
 (2,
  '-0.496*"image" + -0.342*"network" + -0.303*"layer" + 0.279*"model" + -0.277*"function"'),
 (3,
  '-0.359*"value" + 0.340*"learning" + -0.290*"function" + 0.273*"image" + -0.230*"variable"'),
 (4,
  '0.331*"image" + -0.300*"learning" + 0.267*"file" + 0.261*"code" + -0.208*"network"'),
 (5,
  '0.769*"word" + 0.223*"vector" + 0.198*"text" + -0.162*"image" + -0.157*"learning"'),
 (6,
  '-0.479*"function" + 0.387*"image" + 0.375*"feature" + -0.224*"learning" + -0.170*"model"'),
 (7,
  '-0.628*"feature" + -0.266*"learning" + 0.234*"image" + 0.198*"distribution" + 0.182*"probability"'),
 (8,
  '-0.395*"learning" + -0.343*"image" + 0.288*"network" + 0.272*"layer" + 0.228*"feature"'),
 (9,
  '0.494*"ai" + 0.338*"function" + -0.221*"network" + -0.214*"time" + 0.166*"think"')]

In [16]:
jobs_listings_lsamodel = LsiModel(jobs_listings_corpus, num_topics=10, id2word = jobs_listings_dictionary)

In [17]:
jobs_listings_lsamodel.print_topics(num_words=5)

[(0,
  '0.192*"year" + 0.174*"science" + 0.165*"development" + 0.152*"analysis" + 0.139*"skill"'),
 (1,
  '-0.245*"learning" + -0.203*"analytics" + -0.198*"machine" + 0.194*"information" + -0.173*"model"'),
 (2,
  '0.321*"equivalent" + 0.227*"education" + 0.209*"g" + 0.197*"science" + 0.176*"level"'),
 (3,
  '0.243*"quantum" + 0.199*"learning" + 0.196*"machine" + -0.172*"analysis" + 0.170*"world"'),
 (4,
  '0.342*"research" + -0.258*"year" + 0.250*"learning" + 0.187*"machine" + 0.183*"science"'),
 (5,
  '-0.244*"development" + 0.237*"information" + 0.163*"employee" + 0.163*"application" + -0.159*"research"'),
 (6,
  '-0.308*"research" + 0.253*"laboratory" + -0.188*"project" + 0.188*"medical" + -0.184*"development"'),
 (7,
  '0.265*"analytics" + -0.246*"learning" + 0.226*"analysis" + -0.211*"development" + -0.180*"laboratory"'),
 (8,
  '0.475*"product" + -0.207*"laboratory" + 0.194*"development" + -0.187*"service" + 0.152*"engineering"'),
 (9,
  '-0.388*"research" + 0.316*"product" + -0

## LDA Model

In [18]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [19]:
articles_ldamodel = gensim.models.LdaMulticore(articles_corpus, num_topics=10, id2word=articles_dictionary)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
articles_ldamodel.print_topics(num_words=5)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.010*"image" + 0.006*"model" + 0.004*"time" + 0.004*"ai" + 0.004*"value"'),
 (1,
  '0.008*"model" + 0.005*"time" + 0.005*"need" + 0.004*"way" + 0.004*"value"'),
 (2,
  '0.018*"model" + 0.007*"word" + 0.007*"value" + 0.006*"network" + 0.006*"time"'),
 (3,
  '0.014*"model" + 0.007*"learning" + 0.005*"time" + 0.004*"need" + 0.004*"result"'),
 (4,
  '0.014*"model" + 0.006*"learning" + 0.006*"function" + 0.005*"time" + 0.005*"number"'),
 (5,
  '0.009*"model" + 0.006*"learning" + 0.005*"function" + 0.005*"time" + 0.005*"code"'),
 (6,
  '0.013*"model" + 0.008*"learning" + 0.005*"time" + 0.005*"value" + 0.005*"machine"'),
 (7,
  '0.009*"model" + 0.008*"value" + 0.006*"need" + 0.006*"code" + 0.006*"function"'),
 (8,
  '0.008*"model" + 0.006*"value" + 0.006*"time" + 0.005*"feature" + 0.005*"code"'),
 (9,
  '0.008*"learning" + 0.008*"value" + 0.007*"function" + 0.006*"model" + 0.006*"time"')]

In [21]:
pyLDAvis.gensim_models.prepare(articles_ldamodel, articles_corpus, articles_dictionary)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.007111 -0.016184       1        1  16.428754
2      0.016965  0.009970       2        1  15.851344
0     -0.020046  0.006996       3        1  13.939498
8     -0.006085 -0.010483       4        1  11.646561
6      0.006796  0.005459       5        1  10.400992
5     -0.002403 -0.001011       6        1   7.182924
4      0.000139  0.001282       7        1   6.965452
9      0.002700  0.000304       8        1   6.632656
1     -0.006944  0.002825       9        1   6.076694
3      0.001766  0.000841      10        1   4.875124, topic_info=          Term           Freq          Total Category  logprob  loglift
317      model  256477.000000  256477.000000  Default  30.0000  30.0000
103   learning  121168.000000  121168.000000  Default  29.0000  29.0000
196       time  119304.000000  119304.000000  Default  28.0000  28.0000
1064     image   97394.000000   97394.000000  Default  27.0000  27.0000
171    science   49929.000000   49929.000000  Default  26.0000  26.0000
...        ...            ...            ...      ...      ...      ...
211        way    3050.449840   72546.617925  Topic10  -5.9330  -0.1479
375   training    2894.143512   64208.943546  Topic10  -5.9856  -0.0784
74    function    3172.369820  109830.577207  Topic10  -5.8938  -0.5234
26        case    2851.199502   61613.705492  Topic10  -6.0006  -0.0521
1064     image    2977.699112   97394.460497  Topic10  -5.9572  -0.4666

[940 rows x 6 columns], token_table=        Topic      Freq      Term
term                             
104837      7  0.467552     abhay
218         1  0.094520  accuracy
218         2  0.274135  accuracy
218         3  0.099951  accuracy
218         4  0.076750  accuracy
...       ...       ...       ...
59857       9  0.428192        ℙb
48238       1  0.127243        ℙx
48238       2  0.127243        ℙx
48238       4  0.127243        ℙx
48238       8  0.508973        ℙx

[4526 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 3, 1, 9, 7, 6, 5, 10, 2, 4])

In [22]:
jobs_listings_ldamodel = gensim.models.LdaMulticore(jobs_listings_corpus, num_topics=10, id2word=jobs_listings_dictionary)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
jobs_listings_ldamodel.print_topics(num_words=5)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.006*"year" + 0.005*"analysis" + 0.005*"science" + 0.005*"skill" + 0.005*"development"'),
 (1,
  '0.007*"year" + 0.006*"analysis" + 0.005*"development" + 0.005*"technology" + 0.004*"knowledge"'),
 (2,
  '0.007*"technology" + 0.006*"year" + 0.005*"client" + 0.005*"skill" + 0.005*"solution"'),
 (3,
  '0.007*"product" + 0.006*"development" + 0.006*"science" + 0.006*"year" + 0.005*"technical"'),
 (4,
  '0.008*"science" + 0.007*"year" + 0.007*"analysis" + 0.006*"skill" + 0.005*"development"'),
 (5,
  '0.006*"statistical" + 0.005*"learning" + 0.005*"research" + 0.005*"product" + 0.005*"science"'),
 (6,
  '0.006*"year" + 0.006*"analysis" + 0.005*"solution" + 0.005*"analytics" + 0.005*"skill"'),
 (7,
  '0.006*"service" + 0.005*"knowledge" + 0.005*"year" + 0.005*"technology" + 0.005*"development"'),
 (8,
  '0.009*"year" + 0.007*"development" + 0.006*"science" + 0.006*"process" + 0.005*"project"'),
 (9,
  '0.006*"science" + 0.006*"year" + 0.005*"product" + 0.005*"model" + 0.005*"analyti

In [24]:
pyLDAvis.gensim_models.prepare(jobs_listings_ldamodel, jobs_listings_corpus, jobs_listings_dictionary)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.002616  0.003356       1        1  16.551292
9      0.002031 -0.002678       2        1  16.492154
3      0.002195 -0.003590       3        1  11.529469
8      0.006129  0.010378       4        1  10.419787
6     -0.003153  0.003953       5        1  10.133543
0     -0.002776  0.005165       6        1   9.765039
5      0.012984 -0.008142       7        1   9.147523
7     -0.007415  0.001304       8        1   6.516123
1     -0.003772 -0.000596       9        1   5.336596
2     -0.008839 -0.009149      10        1   4.108474, topic_info=            Term         Freq        Total Category  logprob  loglift
311   technology  4318.000000  4318.000000  Default  30.0000  30.0000
204         year  6557.000000  6557.000000  Default  29.0000  29.0000
352       client  2665.000000  2665.000000  Default  28.0000  28.0000
8       analysis  4978.000000  4978.000000  Default  27.0000  27.0000
265    knowledge  3571.000000  3571.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
523  information   122.672941  3708.720507  Topic10  -5.8428  -0.2168
484  application   111.850705  2841.987203  Topic10  -5.9352  -0.0430
662      support   119.363529  3630.515185  Topic10  -5.8702  -0.2228
265    knowledge   116.113266  3571.351377  Topic10  -5.8978  -0.2340
627        model   117.097640  3773.418385  Topic10  -5.8893  -0.2806

[838 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
38699      3  0.590447    abilitynice
6216       2  0.319644      accenture
6216       3  0.152873      accenture
6216       4  0.166771      accenture
6216       5  0.027795      accenture
...      ...       ...            ...
18748      5  0.547719        名bd实习若干
18753      5  0.547392    岗位operation
18754      5  0.547676      工作ead恐被取消
18760      5  0.547224  确认实习企业及职业规划方向
18761      5  0.547442  确认实习意向签订合约并付款

[2250 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 10, 4, 9, 7, 1, 6, 8, 2, 3])